In [1]:
from sklearn import svm, preprocessing, metrics
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN

In [2]:
import pandas as pd
import numpy as np
import os
from collections import Counter
import random

In [3]:
def AU(n):
    # return AU column name
    return f'AU{str(n).zfill(2)}_r'

def to_csv_helper(data, filename, overwrite = False):
    if not overwrite and os.path.exists(filename):
        print('file already exists')
        return 0
    data.to_csv(filename, index=False)
    return 0


In [4]:
# define useful functions

count = 0
RANGE = 1500
def closest_frame(df, f):
    f_near = f
    for x in range(50):
        while len(face_data.loc[face_data['frame'] == f_near, 'milis'].values) == 0:
            f_near -= 1
    print("number of frames retraced: " + str(f - f_near))
    return f_near

def df_near(df, frame, ms_range=RANGE):
    """return rows where milis is within the search range"""
    if frame is None:
        return None
    ms = df.loc[df['frame'] == closest_frame(df,frame), 'milis'].values[0]
    locs = df.loc[(df['milis'] >= ms - ms_range) & (df['milis'] <= ms + ms_range)]
#     locs = df.loc[(df['milis'] >= ms - ms_range) & (df['milis'] <= ms)]
    return locs

def max_df(d):
    if d is None:
        return None
    return pd.DataFrame(d.max()).transpose()

def min_df(d):
    if d is None:
        return None
    return pd.DataFrame(d.min()).transpose()

def mean_df(d):
    if d is None:
        return None
    return pd.DataFrame(d.mean()).transpose()

def sd_df(d):
    if d is None:
        return None
    return pd.DataFrame(d.std()).transpose()

In [5]:
# gather data

# choose which action units to look at
# AUs_to_look_at = [9, 14, 15, 17, 20, 23] pilot 1 and 2
AUs_to_look_at = [1,9,10,14,15,17,20,23]
Excluded_p = ['P6109bd72fa4cdd8f9585cc43','P60e7a66b2cb32cc90fade4e6']

def get_pid(filename):
    """ get pid from dirname """
    return filename.split('_')[0][1:]
def get_mode(filename):
    """ get mode from dirname """
    return int(file.split('_')[2][1])

up_data_max = []
down_data_max = []
up_data_min = []
down_data_min = []
all_data = []

up_data_sd = []
down_data_sd = []

up_data_mean = []
down_data_mean = []

all_data_max = []
all_data_min = []
all_data_mean = []
all_data_sd = []

signals = pd.read_csv('../all_pilots/signals_all_pilots.csv')[['player_id', 'mode', 'signal_type', 'frame_number']]

PATH = "/Users/yoony/Desktop/si_research/face_analysis/implicit_feedback_si_faceanalysis/processed_merged"
csvs = []
for file in sorted(os.listdir(PATH)):
    if file[0] == '.' or file.split('_')[0] in Excluded_p:
        print('excluded:' + file)
        continue
    print(file)
#     csvs.append(pd.read_csv(os.path.join(PATH + "/" + file, 'webcam/aggregated_webcam.csv')))
    data = pd.read_csv(os.path.join(PATH + "/" + file, 'webcam/aggregated_webcam.csv'))
    face_data = data.drop(columns=list(data.columns.values[2:678]))
    face_data = face_data.drop(columns=list(face_data.columns.values[19:]))
#     face_data = face_data[(['frame', 'milis'] + [AU(x) for x in AUs_to_look_at])]
    
    s1 = signals.loc[(signals['player_id'] == get_pid(file)) & (signals['mode'] == get_mode(file))]
    s_up = s1.loc[s1['signal_type'] == 'up'].frame_number.values
    s_down = s1.loc[s1['signal_type'] == 'down'].frame_number.values
    
    print('hi')
    
    for x in s_up:
        print(x)
        up_locs = df_near(face_data, x)
        
        u_max = max_df(up_locs)
        u_min = min_df(up_locs)
        u_sd = sd_df(up_locs)
        u_mean = mean_df(up_locs)
        
        up_data_max.append(u_max)
        up_data_min.append(u_min)
        up_data_sd.append(u_sd)
        up_data_mean.append(u_mean)
        
#         all_data_max.append(u_max)
#         all_data_min.append(u_min)
#         all_data_sd.append(u_sd)
#         all_data_mean.append(u_mean)
        
    for x in s_down:
        down_locs = df_near(face_data, x)
        
        d_max = max_df(down_locs)
        d_min = min_df(down_locs)
        d_sd = sd_df(down_locs)
        d_mean = mean_df(down_locs)
        
        down_data_max.append(d_max)
        down_data_min.append(d_min)
        down_data_sd.append(d_sd)
        down_data_mean.append(d_mean)
        
#         all_data_max.append(d_max)
#         all_data_min.append(d_min)
#         all_data_sd.append(d_sd)
#         all_data_mean.append(d_mean)
        
#     all_data.append(face_data)
    
    

excluded:.DS_Store
P5ee6d2494fd04045b988a12d_voff_m1_g1_t2021_7_10_0_47
hi
165
number of frames retraced: 2
480
number of frames retraced: 1
708
number of frames retraced: 1
1145
number of frames retraced: 2
1621
number of frames retraced: 2
2530
number of frames retraced: 3
2922
number of frames retraced: 3
3115
number of frames retraced: 0
P5ee6d2494fd04045b988a12d_voff_m2_g2_t2021_7_10_0_50
hi
150
number of frames retraced: 0
376
number of frames retraced: 2
914
number of frames retraced: 0
2342
number of frames retraced: 3
P5ef33c05f4c5240413ef9ac2_voff_m1_g1_t2021_7_13_20_38
hi
219
number of frames retraced: 1
893
number of frames retraced: 1
1474
number of frames retraced: 3
2830
number of frames retraced: 0
number of frames retraced: 3
number of frames retraced: 1
number of frames retraced: 2
number of frames retraced: 1
number of frames retraced: 1
P5ef33c05f4c5240413ef9ac2_voff_m2_g2_t2021_7_13_20_34
hi
213
number of frames retraced: 3
570
number of frames retraced: 3
857
numb

In [6]:
def rename_columns(df, tag):
    a = df.copy()
    a.columns = list(map(lambda x: x + tag, a.columns))
    return a

def merge_dfs(dfs):
    a = dfs[0]
    for d in dfs[1:]:
        a = a.join(d, how='outer')
    return a
        

In [7]:
u1 = rename_columns(pd.concat(up_data_max).drop(columns=['frame', 'milis']), '_max').reset_index(drop=True)
u2 = rename_columns(pd.concat(up_data_min).drop(columns=['frame', 'milis']), '_min').reset_index(drop=True)
u3 = rename_columns(pd.concat(up_data_mean).drop(columns=['frame', 'milis']), '_mean').reset_index(drop=True)
u4 = rename_columns(pd.concat(up_data_sd).drop(columns=['frame', 'milis']), '_sd').reset_index(drop=True)

In [8]:
# ups = merge_dfs([u1, u2, u3, u4])
ups = merge_dfs([u1,u2,u3,u4])
ups['type'] = 'up'

In [9]:
d1 = rename_columns(pd.concat(down_data_max).drop(columns=['frame', 'milis']), '_max').reset_index(drop=True)
d2 = rename_columns(pd.concat(down_data_min).drop(columns=['frame', 'milis']), '_min').reset_index(drop=True)
d3 = rename_columns(pd.concat(down_data_mean).drop(columns=['frame', 'milis']), '_mean').reset_index(drop=True)
d4 = rename_columns(pd.concat(down_data_sd).drop(columns=['frame', 'milis']), '_sd').reset_index(drop=True)

In [10]:
# downs = merge_dfs([d1, d2, d3, d4])
downs = merge_dfs([d1,d2,d3,d4])
downs['type'] = 'down'

In [22]:
means = pd.DataFrame(pd.concat(up_data_max).mean(), columns=['up_mean'])
means['down_mean'] = pd.concat(down_data_max).mean()
means['mean_diff'] = means['up_mean'] - means['down_mean']
# means['up_sd'] = pd.concat(up_data_max).std()
# means['down_sd'] = pd.concat(down_data_max).std()
# means['sd_diff'] = means['up_sd'] - means['down_sd']
means = means.drop(index=['frame', 'milis'])
# means['total'] = pd.concat(all_data).mean
# means.to_csv('face/au_data.csv')
means

,up_mean,down_mean,mean_diff
AU01_r,1.227550,1.056522,0.171028
AU02_r,0.957711,0.948841,0.008870
AU04_r,0.866867,0.778986,0.087882
AU05_r,0.934257,0.890870,0.043387
AU06_r,0.153936,0.135362,0.018573
AU07_r,0.481526,0.386812,0.094715
AU09_r,0.427229,0.449565,-0.022336
AU10_r,0.247791,0.359130,-0.111339
AU12_r,0.138916,0.158696,-0.019780
AU14_r,0.394056,0.345507,0.048549


In [12]:
# check number of signals
len(up_data_max) + len(down_data_max)

318

In [18]:
#clean up and finalize the data
signal_data = pd.concat([ups, downs]).reset_index(drop=True)
filename = f'face_data/all_pilots/two_classes/fd_{RANGE}_all.csv'
to_csv_helper(signal_data, filename, overwrite=True)
# signal_data.to_csv(f'face_data/pilot3/two_classes/fd_{RANGE}.csv', index=False)
# signal_types = signal_data['type'].values
# signal_data = signal_data.drop(columns=['type'])

0